# CartPole Environment

In [ ]:
# |export
import gym
import torch.optim as optim
from torch.optim import Adam
from d3rlpy.algos import DQN, DoubleDQN
from d3rlpy.models.optimizers import OptimizerFactory
from d3rlpy.online.buffers import ReplayBuffer
from d3rlpy.online.explorers import LinearDecayEpsilonGreedy, ConstantEpsilonGreedy,NormalNoise

# Simple DQN

load pre-built environment with openAI Gym

first of all , we start by initializing the environment, which is the CartPole environment in this notebook, then we choose : 
- An optimizer that will update the network weights during training(optim_factory). There are a bunch of optimizers that we can select. [View](https://d3rlpy.readthedocs.io/en/stable/references/optimizers.html) 
- [Replay buffer](https://d3rlpy.readthedocs.io/en/stable/references/generated/d3rlpy.online.buffers.ReplayBuffer.html?highlight=ReplayBuffer)  where we can store experiences.
- The explorer, will deal with the exploration/exploitation dilemma, here we can find 3 possible explorers ([Constant explorer](https://d3rlpy.readthedocs.io/en/stable/references/generated/d3rlpy.online.explorers.ConstantEpsilonGreedy.html#d3rlpy.online.explorers.ConstantEpsilonGreedy), [LinearDecayEpsilonGreedy](https://d3rlpy.readthedocs.io/en/stable/references/generated/d3rlpy.online.explorers.LinearDecayEpsilonGreedy.html#d3rlpy.online.explorers.LinearDecayEpsilonGreedy), [NormalNoise](https://d3rlpy.readthedocs.io/en/stable/references/generated/d3rlpy.online.explorers.NormalNoise.html#d3rlpy.online.explorers.NormalNoise))
- The model, here we used the [DQN](https://d3rlpy.readthedocs.io/en/stable/references/generated/d3rlpy.algos.DQN.html#d3rlpy.algos.DQN) in which we can fix multiple hyper-parameters like : the batch size, learning rate, the interval to synchronize the target network, number of steps, the optimizer, the [network architecture](https://d3rlpy.readthedocs.io/en/stable/references/network_architectures.html) (encoder_factory), [scaler function](https://d3rlpy.readthedocs.io/en/stable/references/preprocessing.html) ( here we can set up a custom scaler to preprocess observations, we have to extend the Scaler class and respect the following structure) [View](https://d3rlpy.readthedocs.io/en/v0.91/_modules/d3rlpy/preprocessing/scalers.html)


Once we have setting up the model, it's time to fit it so we will call the fit_online function which take the following step:
1. [Init a replay buffer in case we didn't specify](https://github.com/takuseno/d3rlpy/blob/master/d3rlpy/algos/base.py). 
2. [Start training](https://github.com/takuseno/d3rlpy/blob/d6e87a2dd042653be8c8689568cb29ce59a92bf6/d3rlpy/online/iterators.py#L99) in which we start by : 
    1. Init loggers
    2. Init algorithm params ( fit scaler ,fit action scaler, setup algo)
    3. verify the shape of the observation space , if 3D we are dealing with images and it will apply the 
    [stacking operation](https://github.com/takuseno/d3rlpy/blob/d6e87a2dd042653be8c8689568cb29ce59a92bf6/d3rlpy/preprocessing/stack.py#L40) ( (1,84,84) to (n,84,84) with n = the number of frames to stack ) 
    4. Setup evaluation scorer
    5. get the first observation, and init rollout return(episode return) to 0
    6. Start Training loop :
       - Case image dataset : we add the observation to the stack
       - 2nd case: the observation to float
       - we perform the exploration/exploitation action using the chosen explorer:
           - we reshape the observation (numpy-ndarray) from (n,84,84) to (1,n,84,84) (numpy ndarray)
           - predict the [action](https://github.com/takuseno/d3rlpy/blob/d6e87a2dd042653be8c8689568cb29ce59a92bf6/d3rlpy/online/explorers.py#L28) , [then](https://github.com/takuseno/d3rlpy/blob/v1.1.1/d3rlpy/algos/base.py), [then transformed to tensor and apply transformer](https://github.com/takuseno/d3rlpy/blob/v1.1.1/d3rlpy/torch_utility.py)
           - step environment ( get next observation, reward) and update the rollout return
           - store observation to buffer
           - sample mini batch 
           - update params
       



For offline training:

1. execute the [fitter function](https://github.com/takuseno/d3rlpy/blob/v1.1.1/d3rlpy/base.py#L428)  
2. using the provided MDPDataset, we gather transitions (episodes) 
3. setting up an iterator ( batch)
4. Init logger
5. Init scalers (transformers)
6. Init algo Implemntation
7. [process observation](https://github.com/takuseno/d3rlpy/blob/1ac85b9955408b5e3e9f67ec6592828d8021885b/d3rlpy/base.py#L728) (case image stacking) 
8. Init eval metrics
9. training loop : 
    - use the iterator to get transitions
    - update params with mini-batch [view](https://github.com/takuseno/d3rlpy/blob/1ac85b9955408b5e3e9f67ec6592828d8021885b/d3rlpy/base.py#L738) using the DQN algo [view](https://github.com/takuseno/d3rlpy/blob/1ac85b9955408b5e3e9f67ec6592828d8021885b/d3rlpy/algos/dqn.py#L129). Here we transform observations to tensors [view](https://github.com/takuseno/d3rlpy/blob/1ac85b9955408b5e3e9f67ec6592828d8021885b/d3rlpy/torch_utility.py#L260)

        

In [ ]:
# setup environment
# training env
env = gym.make('CartPole-v0')
# evaluation env
eval_env = gym.make('CartPole-v0')

Initially, we will use Simple DQN algo, and fixing batch size as 32 and synchronize the target network each 100 iterations

In [ ]:
# modify weight decay
optim_factory = OptimizerFactory(Adam, weight_decay=1e-4)
# setup algorithm
dqn = DQN(batch_size=32, # number of batches
          learning_rate=2.5e-4, # learning rate
          target_update_interval=100, # interval to synchronize the target network
          n_steps=1, # N-step TD calculation
          optim_factory= optim_factory # optimizer
         )

Setup a buffer that will save experiences

In [ ]:
# setup replay buffer
buffer = ReplayBuffer(maxlen=1000000, env=env)

Then, We will setup an explorer to deal with the exploration/exploitation problem.
Here, we can use :
    
    - Epsilon Greedy : the agent starts with 100% of exploration and end with 10%
    
    - Constant Epsilon Greedy : a fixed percentage of exploration
    
    - Normal Noise : Normal Noise Explorer

In [ ]:
# setup explorers
explorer = LinearDecayEpsilonGreedy(start_epsilon=1.0,
                                    end_epsilon=0.1,
                                    duration=10000)
#explorer = ConstantEpsilonGreedy(0.3)
#explorer = NormalNoise(mean=0, std=0.1)

Train the simple DQN

In [ ]:
dqn.fit_online(
    env,  # environment
    buffer,  # buffer
    explorer=explorer,  # buffer
    eval_env=eval_env,  # eval environment
    # n_epochs=30,
    n_steps_per_epoch=1000,  # the number of steps per epoch.
    update_interval=100,
    eval_epsilon=0.3,
    save_metrics=True,
    tensorboard_dir="runs",
)

2023-01-09 22:36.35 [info     ] Directory is created at d3rlpy_logs/DQN_online_20230109223635
2023-01-09 22:36.35 [debug    ] Building model...
2023-01-09 22:36.35 [debug    ] Model has been built.
2023-01-09 22:36.35 [info     ] Parameters are saved to d3rlpy_logs/DQN_online_20230109223635/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 0.00025, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'weight_decay': 0.0001}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 100, 'use_gpu': None, 'algorithm': 'DQN', 'observation_shape': (4,), 'action_size': 2}


  0%|          | 0/1000000 [00:00<?, ?it/s]

2023-01-09 22:36.35 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_1000.pt
2023-01-09 22:36.35 [info     ] DQN_online_20230109223635: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.00018756651878356934, 'time_environment_step': 1.2215137481689454e-05, 'time_step': 0.00022780752182006837, 'rollout_return': 27.555555555555557, 'time_sample_batch': 5.5742263793945315e-05, 'time_algorithm_update': 0.0013378620147705077, 'loss': 0.45788215696811674, 'evaluation': 10.1} step=1000
2023-01-09 22:36.35 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_2000.pt
2023-01-09 22:36.35 [info     ] DQN_online_20230109223635: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.00017923951148986816, 'time_environment_step': 1.2298345565795898e-05, 'time_step': 0.00021900510787963866, 'rollout_return': 16.74576271186441, 'time_sample_batch': 6.020069122314453e-05, 'time_algorithm_update': 0.0012368440628051757, 'loss': 0

2023-01-09 22:36.39 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_17000.pt
2023-01-09 22:36.39 [info     ] DQN_online_20230109223635: epoch=17 step=17000 epoch=17 metrics={'time_inference': 0.00018497991561889648, 'time_environment_step': 1.254892349243164e-05, 'time_step': 0.0002299017906188965, 'rollout_return': 10.923076923076923, 'time_sample_batch': 8.761882781982422e-05, 'time_algorithm_update': 0.0016250848770141602, 'loss': 0.06535573452711105, 'evaluation': 11.8} step=17000
2023-01-09 22:36.39 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_18000.pt
2023-01-09 22:36.39 [info     ] DQN_online_20230109223635: epoch=18 step=18000 epoch=18 metrics={'time_inference': 0.00017815589904785156, 'time_environment_step': 1.2163639068603516e-05, 'time_step': 0.00021811413764953613, 'rollout_return': 12.419753086419753, 'time_sample_batch': 6.601810455322265e-05, 'time_algorithm_update': 0.0012514114379882813, 'l

2023-01-09 22:36.43 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_33000.pt
2023-01-09 22:36.43 [info     ] DQN_online_20230109223635: epoch=33 step=33000 epoch=33 metrics={'time_inference': 0.0001876828670501709, 'time_environment_step': 1.2257575988769531e-05, 'time_step': 0.00022875380516052245, 'rollout_return': 27.11111111111111, 'time_sample_batch': 7.593631744384766e-05, 'time_algorithm_update': 0.0013556957244873046, 'loss': 0.20546391755342483, 'evaluation': 55.9} step=33000
2023-01-09 22:36.43 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_34000.pt
2023-01-09 22:36.43 [info     ] DQN_online_20230109223635: epoch=34 step=34000 epoch=34 metrics={'time_inference': 0.00017185258865356446, 'time_environment_step': 1.166248321533203e-05, 'time_step': 0.00020886659622192382, 'rollout_return': 65.86666666666666, 'time_sample_batch': 6.380081176757813e-05, 'time_algorithm_update': 0.0011276721954345704, 'los

...

2023-01-09 22:42.58 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_929000.pt
2023-01-09 22:42.58 [info     ] DQN_online_20230109223635: epoch=929 step=929000 epoch=929 metrics={'time_inference': 0.00018852043151855468, 'time_environment_step': 1.2020349502563477e-05, 'time_step': 0.0002460746765136719, 'rollout_return': 191.83333333333334, 'time_sample_batch': 7.500648498535157e-05, 'time_algorithm_update': 0.0031226396560668944, 'loss': 0.4890184126794338, 'evaluation': 141.6} step=929000
2023-01-09 22:42.58 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_930000.pt
2023-01-09 22:42.58 [info     ] DQN_online_20230109223635: epoch=930 step=930000 epoch=930 metrics={'time_inference': 0.0001873586177825928, 'time_environment_step': 1.1795282363891601e-05, 'time_step': 0.00024559545516967776, 'time_sample_batch': 9.04083251953125e-05, 'time_algorithm_update': 0.003199458122253418, 'loss': 0.5710880764760077, 'roll

2023-01-09 22:43.05 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_945000.pt
2023-01-09 22:43.05 [info     ] DQN_online_20230109223635: epoch=945 step=945000 epoch=945 metrics={'time_inference': 0.00019880890846252442, 'time_environment_step': 1.2511253356933594e-05, 'time_step': 0.0002622816562652588, 'time_sample_batch': 0.00010004043579101563, 'time_algorithm_update': 0.003546285629272461, 'loss': 0.5257206964306533, 'rollout_return': 178.2, 'evaluation': 149.2} step=945000
2023-01-09 22:43.06 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_946000.pt
2023-01-09 22:43.06 [info     ] DQN_online_20230109223635: epoch=946 step=946000 epoch=946 metrics={'time_inference': 0.00021265125274658203, 'time_environment_step': 1.3644218444824219e-05, 'time_step': 0.0002785642147064209, 'rollout_return': 165.57142857142858, 'time_sample_batch': 0.00010597705841064453, 'time_algorithm_update': 0.003551340103149414, 'loss'

2023-01-09 22:43.13 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_961000.pt
2023-01-09 22:43.13 [info     ] DQN_online_20230109223635: epoch=961 step=961000 epoch=961 metrics={'time_inference': 0.00020440101623535157, 'time_environment_step': 1.250910758972168e-05, 'time_step': 0.0002656097412109375, 'rollout_return': 177.16666666666666, 'time_sample_batch': 9.737014770507812e-05, 'time_algorithm_update': 0.003342151641845703, 'loss': 0.4362704297527671, 'evaluation': 178.5} step=961000
2023-01-09 22:43.13 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_962000.pt
2023-01-09 22:43.13 [info     ] DQN_online_20230109223635: epoch=962 step=962000 epoch=962 metrics={'time_inference': 0.00018666315078735352, 'time_environment_step': 1.1761903762817382e-05, 'time_step': 0.0002429778575897217, 'rollout_return': 178.0, 'time_sample_batch': 7.586479187011718e-05, 'time_algorithm_update': 0.0030449628829956055, 'loss': 

2023-01-09 22:43.20 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_977000.pt
2023-01-09 22:43.20 [info     ] DQN_online_20230109223635: epoch=977 step=977000 epoch=977 metrics={'time_inference': 0.00019909214973449707, 'time_environment_step': 1.2601613998413085e-05, 'time_step': 0.00025957322120666505, 'rollout_return': 184.16666666666666, 'time_sample_batch': 7.963180541992188e-05, 'time_algorithm_update': 0.0032491445541381835, 'loss': 0.09290997460484504, 'evaluation': 171.9} step=977000
2023-01-09 22:43.21 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_978000.pt
2023-01-09 22:43.21 [info     ] DQN_online_20230109223635: epoch=978 step=978000 epoch=978 metrics={'time_inference': 0.00019678974151611327, 'time_environment_step': 1.2453556060791016e-05, 'time_step': 0.0002572653293609619, 'time_sample_batch': 7.953643798828126e-05, 'time_algorithm_update': 0.003311443328857422, 'loss': 0.35147553142160176, '

2023-01-09 22:43.28 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_993000.pt
2023-01-09 22:43.28 [info     ] DQN_online_20230109223635: epoch=993 step=993000 epoch=993 metrics={'time_inference': 0.0001863868236541748, 'time_environment_step': 1.1698722839355468e-05, 'time_step': 0.00024347996711730958, 'time_sample_batch': 7.195472717285157e-05, 'time_algorithm_update': 0.0031319856643676758, 'loss': 0.6063910726457834, 'rollout_return': 183.4, 'evaluation': 163.4} step=993000
2023-01-09 22:43.28 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109223635/model_994000.pt
2023-01-09 22:43.28 [info     ] DQN_online_20230109223635: epoch=994 step=994000 epoch=994 metrics={'time_inference': 0.0001854395866394043, 'time_environment_step': 1.224660873413086e-05, 'time_step': 0.00024364304542541504, 'time_sample_batch': 7.46011734008789e-05, 'time_algorithm_update': 0.0031879663467407225, 'loss': 0.6674715025350452, 'rollout_return': 

# DQN Using N-steps and Noisy Networks

In [ ]:
# setup environment
# training env
env = gym.make('CartPole-v0')
# evaluation env
eval_env = gym.make('CartPole-v0')

In [ ]:
# modify weight decay
optim_factory = OptimizerFactory(Adam, weight_decay=1e-4)
# setup algorithm
dqn = DQN(batch_size=32, # number of batches
          learning_rate=2.5e-4, # learning rate
          target_update_interval=100, # interval to synchronize the target network
          n_steps=4, # N-step TD calculation
          optim_factory= optim_factory # optimizer
         )

In [ ]:
# setup replay buffer
buffer = ReplayBuffer(maxlen=1000000, env=env)

In [ ]:
explorer = LinearDecayEpsilonGreedy(start_epsilon=1.0,
                                    end_epsilon=0.1,
                                    duration=10000)
#explorer = NormalNoise(mean= 0, std=0.1)

In [ ]:
dqn.fit_online(
    env,  # environment
    buffer,  # buffer
    explorer=explorer,  # buffer
    eval_env=eval_env,  # eval environment
    n_steps_per_epoch=1000,  # the number of steps per epoch.
    update_interval=100,
    eval_epsilon=0.3,
    save_metrics=True,
    tensorboard_dir="runs",
)

2023-01-09 22:45.41 [info     ] Directory is created at d3rlpy_logs/DQN_online_20230109224541
2023-01-09 22:45.41 [debug    ] Building model...
2023-01-09 22:45.41 [debug    ] Model has been built.
2023-01-09 22:45.41 [info     ] Parameters are saved to d3rlpy_logs/DQN_online_20230109224541/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 0.00025, 'n_critics': 1, 'n_frames': 1, 'n_steps': 4, 'optim_factory': {'optim_cls': 'Adam', 'weight_decay': 0.0001}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 100, 'use_gpu': None, 'algorithm': 'DQN', 'observation_shape': (4,), 'action_size': 2}


  0%|          | 0/1000000 [00:00<?, ?it/s]

2023-01-09 22:45.41 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_1000.pt
2023-01-09 22:45.41 [info     ] DQN_online_20230109224541: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.00017836833000183105, 'time_environment_step': 1.2113809585571288e-05, 'time_step': 0.00021889591217041015, 'rollout_return': 21.88888888888889, 'time_sample_batch': 7.402896881103516e-05, 'time_algorithm_update': 0.001340937614440918, 'loss': 3.1005072593688965, 'evaluation': 11.9} step=1000
2023-01-09 22:45.41 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_2000.pt
2023-01-09 22:45.41 [info     ] DQN_online_20230109224541: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.00017458009719848632, 'time_environment_step': 1.1927604675292968e-05, 'time_step': 0.0002142350673675537, 'rollout_return': 21.847826086956523, 'time_sample_batch': 8.685588836669922e-05, 'time_algorithm_update': 0.0012852668762207032, 'loss': 2.800

2023-01-09 22:45.45 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_17000.pt
2023-01-09 22:45.45 [info     ] DQN_online_20230109224541: epoch=17 step=17000 epoch=17 metrics={'time_inference': 0.0001729733943939209, 'time_environment_step': 1.205897331237793e-05, 'time_step': 0.00021257472038269042, 'rollout_return': 9.97, 'time_sample_batch': 6.797313690185547e-05, 'time_algorithm_update': 0.001219463348388672, 'loss': 1.198813933134079, 'evaluation': 10.5} step=17000
2023-01-09 22:45.45 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_18000.pt
2023-01-09 22:45.45 [info     ] DQN_online_20230109224541: epoch=18 step=18000 epoch=18 metrics={'time_inference': 0.00017379117012023926, 'time_environment_step': 1.2032032012939453e-05, 'time_step': 0.0002145075798034668, 'rollout_return': 10.309278350515465, 'time_sample_batch': 8.101463317871094e-05, 'time_algorithm_update': 0.0013301849365234375, 'loss': 1.0410839080

2023-01-09 22:45.49 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_33000.pt
2023-01-09 22:45.49 [info     ] DQN_online_20230109224541: epoch=33 step=33000 epoch=33 metrics={'time_inference': 0.00018792557716369629, 'time_environment_step': 1.2913465499877929e-05, 'time_step': 0.00023414850234985352, 'rollout_return': 10.956043956043956, 'time_sample_batch': 8.9263916015625e-05, 'time_algorithm_update': 0.001676487922668457, 'loss': 1.9890812993049622, 'evaluation': 11.6} step=33000
2023-01-09 22:45.49 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_34000.pt
2023-01-09 22:45.49 [info     ] DQN_online_20230109224541: epoch=34 step=34000 epoch=34 metrics={'time_inference': 0.00020075535774230957, 'time_environment_step': 1.326298713684082e-05, 'time_step': 0.0002518947124481201, 'rollout_return': 11.386363636363637, 'time_sample_batch': 0.00010564327239990235, 'time_algorithm_update': 0.002048039436340332, 'loss'

...

2023-01-09 22:52.20 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_929000.pt
2023-01-09 22:52.20 [info     ] DQN_online_20230109224541: epoch=929 step=929000 epoch=929 metrics={'time_inference': 0.00018801641464233397, 'time_environment_step': 1.2455940246582031e-05, 'time_step': 0.0002459297180175781, 'time_sample_batch': 8.003711700439453e-05, 'time_algorithm_update': 0.0031229257583618164, 'loss': 10.175838565826416, 'rollout_return': 192.0, 'evaluation': 191.6} step=929000
2023-01-09 22:52.20 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_930000.pt
2023-01-09 22:52.20 [info     ] DQN_online_20230109224541: epoch=930 step=930000 epoch=930 metrics={'time_inference': 0.00019049906730651855, 'time_environment_step': 1.2019634246826172e-05, 'time_step': 0.00024882054328918455, 'rollout_return': 200.0, 'time_sample_batch': 8.578300476074219e-05, 'time_algorithm_update': 0.0031827688217163086, 'loss': 7.68853971

2023-01-09 22:52.27 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_945000.pt
2023-01-09 22:52.27 [info     ] DQN_online_20230109224541: epoch=945 step=945000 epoch=945 metrics={'time_inference': 0.0002013373374938965, 'time_environment_step': 1.2864112854003907e-05, 'time_step': 0.0002645902633666992, 'rollout_return': 198.5, 'time_sample_batch': 8.997917175292968e-05, 'time_algorithm_update': 0.0034978389739990234, 'loss': 7.626542234420777, 'evaluation': 189.0} step=945000
2023-01-09 22:52.28 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_946000.pt
2023-01-09 22:52.28 [info     ] DQN_online_20230109224541: epoch=946 step=946000 epoch=946 metrics={'time_inference': 0.00020232605934143065, 'time_environment_step': 1.2772321701049804e-05, 'time_step': 0.00026475286483764647, 'time_sample_batch': 8.873939514160156e-05, 'time_algorithm_update': 0.0034337759017944334, 'loss': 5.67203848361969, 'rollout_return': 1

2023-01-09 22:52.35 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_961000.pt
2023-01-09 22:52.35 [info     ] DQN_online_20230109224541: epoch=961 step=961000 epoch=961 metrics={'time_inference': 0.00018981719017028807, 'time_environment_step': 1.1979818344116211e-05, 'time_step': 0.00024959588050842287, 'rollout_return': 194.2, 'time_sample_batch': 8.3160400390625e-05, 'time_algorithm_update': 0.0033428430557250976, 'loss': 8.811341953277587, 'evaluation': 198.3} step=961000
2023-01-09 22:52.36 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_962000.pt
2023-01-09 22:52.36 [info     ] DQN_online_20230109224541: epoch=962 step=962000 epoch=962 metrics={'time_inference': 0.00019053053855895997, 'time_environment_step': 1.1989116668701172e-05, 'time_step': 0.0002504019737243652, 'rollout_return': 200.0, 'time_sample_batch': 8.35418701171875e-05, 'time_algorithm_update': 0.003353738784790039, 'loss': 10.179013895988

2023-01-09 22:52.43 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_977000.pt
2023-01-09 22:52.43 [info     ] DQN_online_20230109224541: epoch=977 step=977000 epoch=977 metrics={'time_inference': 0.00019168233871459962, 'time_environment_step': 1.2108325958251953e-05, 'time_step': 0.000252932071685791, 'time_sample_batch': 8.50677490234375e-05, 'time_algorithm_update': 0.0034669876098632813, 'loss': 7.352555441856384, 'rollout_return': 196.2, 'evaluation': 189.1} step=977000
2023-01-09 22:52.44 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_978000.pt
2023-01-09 22:52.44 [info     ] DQN_online_20230109224541: epoch=978 step=978000 epoch=978 metrics={'time_inference': 0.00018972635269165039, 'time_environment_step': 1.1942148208618164e-05, 'time_step': 0.00025107479095458986, 'rollout_return': 187.4, 'time_sample_batch': 0.00010101795196533203, 'time_algorithm_update': 0.0034870147705078126, 'loss': 6.3159824013

2023-01-09 22:52.51 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_993000.pt
2023-01-09 22:52.51 [info     ] DQN_online_20230109224541: epoch=993 step=993000 epoch=993 metrics={'time_inference': 0.00022044730186462403, 'time_environment_step': 1.3932943344116211e-05, 'time_step': 0.0002915012836456299, 'time_sample_batch': 9.78708267211914e-05, 'time_algorithm_update': 0.0039763212203979496, 'loss': 7.558268451690674, 'rollout_return': 190.2, 'evaluation': 188.0} step=993000
2023-01-09 22:52.52 [info     ] Model parameters are saved to d3rlpy_logs/DQN_online_20230109224541/model_994000.pt
2023-01-09 22:52.52 [info     ] DQN_online_20230109224541: epoch=994 step=994000 epoch=994 metrics={'time_inference': 0.00021530652046203614, 'time_environment_step': 1.361846923828125e-05, 'time_step': 0.00028557586669921873, 'rollout_return': 191.4, 'time_sample_batch': 9.989738464355469e-05, 'time_algorithm_update': 0.00401601791381836, 'loss': 7.0261929988861

# Double DQN

In [ ]:
# setup environment
# training env
env = gym.make('CartPole-v0')
# evaluation env
eval_env = gym.make('CartPole-v0')

In [ ]:
# modify weight decay
optim_factory = OptimizerFactory(Adam, weight_decay=1e-4)
# setup algorithm
ddqn = DoubleDQN(
    batch_size=32,  # number of batches
    learning_rate=2.5e-4,  # learning rate
    target_update_interval=100,  # interval to synchronize the target network
    n_steps=1,  # N-step TD calculation
    optim_factory=optim_factory,  # optimizer
)

In [ ]:
# setup replay buffer
buffer = ReplayBuffer(maxlen=1000000, env=env)

In [ ]:
explorer = LinearDecayEpsilonGreedy(start_epsilon=1.0,
                                    end_epsilon=0.1,
                                    duration=10000)
#explorer = NormalNoise(mean= 0, std=0.1)

In [ ]:
ddqn.fit_online(
    env,  # environment
    buffer,  # buffer
    explorer=explorer,  # buffer
    eval_env=eval_env,  # eval environment
    n_steps_per_epoch=1000,  # the number of steps per epoch.
    update_interval=100,
    eval_epsilon=0.3,
    save_metrics=True,
    tensorboard_dir="runs",
)

2023-01-09 22:54.06 [info     ] Directory is created at d3rlpy_logs/DoubleDQN_online_20230109225406
2023-01-09 22:54.06 [debug    ] Building model...
2023-01-09 22:54.06 [debug    ] Model has been built.
2023-01-09 22:54.06 [info     ] Parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 0.00025, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'weight_decay': 0.0001}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 100, 'use_gpu': None, 'algorithm': 'DoubleDQN', 'observation_shape': (4,), 'action_size': 2}


  0%|          | 0/1000000 [00:00<?, ?it/s]

2023-01-09 22:54.07 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_1000.pt
2023-01-09 22:54.07 [info     ] DoubleDQN_online_20230109225406: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.0001834700107574463, 'time_environment_step': 1.2191295623779297e-05, 'time_step': 0.00022473311424255372, 'rollout_return': 21.456521739130434, 'time_sample_batch': 6.0439109802246094e-05, 'time_algorithm_update': 0.001414179801940918, 'loss': 0.4499122738838196, 'evaluation': 17.9} step=1000
2023-01-09 22:54.07 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_2000.pt
2023-01-09 22:54.07 [info     ] DoubleDQN_online_20230109225406: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.0001723635196685791, 'time_environment_step': 1.1724472045898438e-05, 'time_step': 0.00021134018898010253, 'rollout_return': 22.931818181818183, 'time_sample_batch': 5.958080291748047e-05, 'time_algorithm_update': 0.001273870

2023-01-09 22:54.11 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_17000.pt
2023-01-09 22:54.11 [info     ] DoubleDQN_online_20230109225406: epoch=17 step=17000 epoch=17 metrics={'time_inference': 0.0001748044490814209, 'time_environment_step': 1.2066841125488282e-05, 'time_step': 0.00021494364738464355, 'rollout_return': 12.717948717948717, 'time_sample_batch': 6.334781646728515e-05, 'time_algorithm_update': 0.0013026714324951172, 'loss': 0.05499728135764599, 'evaluation': 13.3} step=17000
2023-01-09 22:54.11 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_18000.pt
2023-01-09 22:54.11 [info     ] DoubleDQN_online_20230109225406: epoch=18 step=18000 epoch=18 metrics={'time_inference': 0.00018171048164367676, 'time_environment_step': 1.2233257293701172e-05, 'time_step': 0.00022275471687316896, 'rollout_return': 11.952380952380953, 'time_sample_batch': 7.030963897705078e-05, 'time_algorithm_update': 

2023-01-09 22:54.14 [info     ] DoubleDQN_online_20230109225406: epoch=32 step=32000 epoch=32 metrics={'time_inference': 0.00017589497566223143, 'time_environment_step': 1.2144565582275391e-05, 'time_step': 0.00021594095230102539, 'rollout_return': 9.92079207920792, 'time_sample_batch': 6.685256958007812e-05, 'time_algorithm_update': 0.0012719869613647462, 'loss': 0.2401246875524521, 'evaluation': 12.5} step=32000
2023-01-09 22:54.15 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_33000.pt
2023-01-09 22:54.15 [info     ] DoubleDQN_online_20230109225406: epoch=33 step=33000 epoch=33 metrics={'time_inference': 0.0001855642795562744, 'time_environment_step': 1.2236595153808593e-05, 'time_step': 0.00022681236267089845, 'rollout_return': 15.80952380952381, 'time_sample_batch': 7.014274597167968e-05, 'time_algorithm_update': 0.0013564825057983398, 'loss': 0.22410463392734528, 'evaluation': 20.5} step=33000
2023-01-09 22:54.15 [info     ] Model para

...

2023-01-09 23:00.41 [info     ] DoubleDQN_online_20230109225406: epoch=938 step=938000 epoch=938 metrics={'time_inference': 0.00018326544761657716, 'time_environment_step': 1.125955581665039e-05, 'time_step': 0.0002449667453765869, 'rollout_return': 178.83333333333334, 'time_sample_batch': 6.852149963378906e-05, 'time_algorithm_update': 0.0036962747573852537, 'loss': 0.7214938316494226, 'evaluation': 165.4} step=938000
2023-01-09 23:00.42 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_939000.pt
2023-01-09 23:00.42 [info     ] DoubleDQN_online_20230109225406: epoch=939 step=939000 epoch=939 metrics={'time_inference': 0.00018968892097473146, 'time_environment_step': 1.1741399765014648e-05, 'time_step': 0.0002522366046905518, 'rollout_return': 168.83333333333334, 'time_sample_batch': 7.290840148925782e-05, 'time_algorithm_update': 0.0036767959594726563, 'loss': 0.8748721834272146, 'evaluation': 160.7} step=939000
2023-01-09 23:00.42 [info     ]

2023-01-09 23:00.48 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_954000.pt
2023-01-09 23:00.48 [info     ] DoubleDQN_online_20230109225406: epoch=954 step=954000 epoch=954 metrics={'time_inference': 0.00019645309448242188, 'time_environment_step': 1.2047767639160155e-05, 'time_step': 0.0002601172924041748, 'rollout_return': 166.0, 'time_sample_batch': 7.479190826416015e-05, 'time_algorithm_update': 0.0037172317504882814, 'loss': 0.44719546642154456, 'evaluation': 174.1} step=954000
2023-01-09 23:00.49 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_955000.pt
2023-01-09 23:00.49 [info     ] DoubleDQN_online_20230109225406: epoch=955 step=955000 epoch=955 metrics={'time_inference': 0.0001872408390045166, 'time_environment_step': 1.1641740798950195e-05, 'time_step': 0.00025026273727416995, 'rollout_return': 165.66666666666666, 'time_sample_batch': 7.176399230957031e-05, 'time_algorithm_update': 0.00

2023-01-09 23:00.55 [info     ] DoubleDQN_online_20230109225406: epoch=969 step=969000 epoch=969 metrics={'time_inference': 0.00019206666946411133, 'time_environment_step': 1.2021780014038086e-05, 'time_step': 0.000256094217300415, 'time_sample_batch': 7.138252258300781e-05, 'time_algorithm_update': 0.0037734508514404297, 'loss': 0.6268476622179151, 'rollout_return': 169.5, 'evaluation': 155.6} step=969000
2023-01-09 23:00.55 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_970000.pt
2023-01-09 23:00.55 [info     ] DoubleDQN_online_20230109225406: epoch=970 step=970000 epoch=970 metrics={'time_inference': 0.00018709588050842284, 'time_environment_step': 1.1591672897338867e-05, 'time_step': 0.00024961614608764646, 'rollout_return': 155.66666666666666, 'time_sample_batch': 7.052421569824219e-05, 'time_algorithm_update': 0.00371248722076416, 'loss': 0.8857485834509135, 'evaluation': 158.1} step=970000
2023-01-09 23:00.56 [info     ] Model paramet

2023-01-09 23:01.02 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_985000.pt
2023-01-09 23:01.02 [info     ] DoubleDQN_online_20230109225406: epoch=985 step=985000 epoch=985 metrics={'time_inference': 0.00018503713607788085, 'time_environment_step': 1.1523246765136719e-05, 'time_step': 0.0002491042613983154, 'rollout_return': 161.14285714285714, 'time_sample_batch': 7.023811340332032e-05, 'time_algorithm_update': 0.0038583993911743162, 'loss': 0.8923199677839875, 'evaluation': 147.7} step=985000
2023-01-09 23:01.02 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109225406/model_986000.pt
2023-01-09 23:01.02 [info     ] DoubleDQN_online_20230109225406: epoch=986 step=986000 epoch=986 metrics={'time_inference': 0.00019034981727600097, 'time_environment_step': 1.1890649795532226e-05, 'time_step': 0.00025533246994018555, 'time_sample_batch': 7.264614105224609e-05, 'time_algorithm_update': 0.003881692886352539, 'loss':

2023-01-09 23:01.09 [info     ] DoubleDQN_online_20230109225406: epoch=1000 step=1000000 epoch=1000 metrics={'time_inference': 0.00019473671913146972, 'time_environment_step': 1.2192249298095703e-05, 'time_step': 0.00026212263107299805, 'time_sample_batch': 7.617473602294922e-05, 'time_algorithm_update': 0.004061436653137207, 'loss': 0.7044335409998894, 'rollout_return': 148.71428571428572, 'evaluation': 133.6} step=1000000


# Double DQN +  Using N-steps and Noisy Networks

In [ ]:
# setup environment
# training env
env = gym.make('CartPole-v0')
# evaluation env
eval_env = gym.make('CartPole-v0')

In [ ]:
# modify weight decay
optim_factory = OptimizerFactory(Adam, weight_decay=1e-4)
# setup algorithm
ddqn = DoubleDQN(
    batch_size=32,  # number of batches
    learning_rate=2.5e-4,  # learning rate
    target_update_interval=100,  # interval to synchronize the target network
    n_steps=4,  # N-step TD calculation
    optim_factory=optim_factory,  # optimizer
)

In [ ]:
# setup replay buffer
buffer = ReplayBuffer(maxlen=1000000, env=env)

In [ ]:
explorer = LinearDecayEpsilonGreedy(start_epsilon=1.0,
                                    end_epsilon=0.1,
                                    duration=10000)

In [ ]:
ddqn.fit_online(
    env,  # environment
    buffer,  # buffer
    explorer=explorer,  # buffer
    eval_env=eval_env,  # eval environment
    n_steps_per_epoch=1000,  # the number of steps per epoch.
    update_interval=100,
    eval_epsilon=0.3,
    save_metrics=True,
    tensorboard_dir="runs",
)

2023-01-09 23:09.13 [info     ] Directory is created at d3rlpy_logs/DoubleDQN_online_20230109230913
2023-01-09 23:09.13 [debug    ] Building model...
2023-01-09 23:09.13 [debug    ] Model has been built.
2023-01-09 23:09.13 [info     ] Parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/params.json params={'action_scaler': None, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 0.00025, 'n_critics': 1, 'n_frames': 1, 'n_steps': 4, 'optim_factory': {'optim_cls': 'Adam', 'weight_decay': 0.0001}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 100, 'use_gpu': None, 'algorithm': 'DoubleDQN', 'observation_shape': (4,), 'action_size': 2}


  0%|          | 0/1000000 [00:00<?, ?it/s]

2023-01-09 23:09.13 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_1000.pt
2023-01-09 23:09.13 [info     ] DoubleDQN_online_20230109230913: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.0001857912540435791, 'time_environment_step': 1.2485265731811523e-05, 'time_step': 0.00022872090339660646, 'rollout_return': 22.0, 'time_sample_batch': 7.038116455078125e-05, 'time_algorithm_update': 0.0014892339706420899, 'loss': 3.0368156671524047, 'evaluation': 12.0} step=1000
2023-01-09 23:09.14 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_2000.pt
2023-01-09 23:09.14 [info     ] DoubleDQN_online_20230109230913: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.00017947816848754881, 'time_environment_step': 1.2160062789916992e-05, 'time_step': 0.00022032904624938964, 'rollout_return': 22.08888888888889, 'time_sample_batch': 7.259845733642578e-05, 'time_algorithm_update': 0.0013334512710571288, 'l

2023-01-09 23:09.18 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_17000.pt
2023-01-09 23:09.18 [info     ] DoubleDQN_online_20230109230913: epoch=17 step=17000 epoch=17 metrics={'time_inference': 0.00018135547637939452, 'time_environment_step': 1.222991943359375e-05, 'time_step': 0.00022182083129882813, 'rollout_return': 29.5, 'time_sample_batch': 7.824897766113281e-05, 'time_algorithm_update': 0.001309037208557129, 'loss': 1.0468122839927674, 'evaluation': 56.1} step=17000
2023-01-09 23:09.18 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_18000.pt
2023-01-09 23:09.18 [info     ] DoubleDQN_online_20230109230913: epoch=18 step=18000 epoch=18 metrics={'time_inference': 0.0001739804744720459, 'time_environment_step': 1.1828184127807616e-05, 'time_step': 0.00021347761154174804, 'rollout_return': 32.61290322580645, 'time_sample_batch': 7.481575012207032e-05, 'time_algorithm_update': 0.0013258695602416

2023-01-09 23:09.22 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_33000.pt
2023-01-09 23:09.22 [info     ] DoubleDQN_online_20230109230913: epoch=33 step=33000 epoch=33 metrics={'time_inference': 0.0001750352382659912, 'time_environment_step': 1.1987924575805664e-05, 'time_step': 0.0002171926498413086, 'rollout_return': 21.020833333333332, 'time_sample_batch': 9.708404541015625e-05, 'time_algorithm_update': 0.001510310173034668, 'loss': 2.4684374928474426, 'evaluation': 23.0} step=33000
2023-01-09 23:09.22 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_34000.pt
2023-01-09 23:09.22 [info     ] DoubleDQN_online_20230109230913: epoch=34 step=34000 epoch=34 metrics={'time_inference': 0.0001757187843322754, 'time_environment_step': 1.2021541595458984e-05, 'time_step': 0.00021554994583129883, 'rollout_return': 22.74418604651163, 'time_sample_batch': 7.646083831787109e-05, 'time_algorithm_update': 0.001

...

2023-01-09 23:16.06 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_923000.pt
2023-01-09 23:16.06 [info     ] DoubleDQN_online_20230109230913: epoch=923 step=923000 epoch=923 metrics={'time_inference': 0.00019301605224609375, 'time_environment_step': 1.2356042861938477e-05, 'time_step': 0.0002504916191101074, 'rollout_return': 200.0, 'time_sample_batch': 8.280277252197266e-05, 'time_algorithm_update': 0.003030991554260254, 'loss': 3.0850295066833495, 'evaluation': 200.0} step=923000
2023-01-09 23:16.06 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_924000.pt
2023-01-09 23:16.06 [info     ] DoubleDQN_online_20230109230913: epoch=924 step=924000 epoch=924 metrics={'time_inference': 0.00019269967079162597, 'time_environment_step': 1.2313127517700196e-05, 'time_step': 0.00025052499771118165, 'rollout_return': 200.0, 'time_sample_batch': 8.71896743774414e-05, 'time_algorithm_update': 0.00307366847991943

2023-01-09 23:16.14 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_939000.pt
2023-01-09 23:16.14 [info     ] DoubleDQN_online_20230109230913: epoch=939 step=939000 epoch=939 metrics={'time_inference': 0.0001855144500732422, 'time_environment_step': 1.186823844909668e-05, 'time_step': 0.00024166369438171386, 'rollout_return': 200.0, 'time_sample_batch': 8.230209350585937e-05, 'time_algorithm_update': 0.0030002355575561523, 'loss': 3.3416054964065554, 'evaluation': 196.1} step=939000
2023-01-09 23:16.14 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_940000.pt
2023-01-09 23:16.14 [info     ] DoubleDQN_online_20230109230913: epoch=940 step=940000 epoch=940 metrics={'time_inference': 0.00018558645248413085, 'time_environment_step': 1.1979818344116211e-05, 'time_step': 0.0002440354824066162, 'rollout_return': 200.0, 'time_sample_batch': 9.953975677490234e-05, 'time_algorithm_update': 0.00318193435668945

2023-01-09 23:16.22 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_955000.pt
2023-01-09 23:16.22 [info     ] DoubleDQN_online_20230109230913: epoch=955 step=955000 epoch=955 metrics={'time_inference': 0.00018744444847106933, 'time_environment_step': 1.2000322341918945e-05, 'time_step': 0.00024498820304870607, 'rollout_return': 200.0, 'time_sample_batch': 9.26971435546875e-05, 'time_algorithm_update': 0.003103470802307129, 'loss': 2.9173078775405883, 'evaluation': 193.8} step=955000
2023-01-09 23:16.22 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_956000.pt
2023-01-09 23:16.22 [info     ] DoubleDQN_online_20230109230913: epoch=956 step=956000 epoch=956 metrics={'time_inference': 0.00018494868278503418, 'time_environment_step': 1.1895179748535157e-05, 'time_step': 0.00024546337127685547, 'rollout_return': 200.0, 'time_sample_batch': 8.664131164550781e-05, 'time_algorithm_update': 0.0034299373626708

2023-01-09 23:16.30 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_971000.pt
2023-01-09 23:16.30 [info     ] DoubleDQN_online_20230109230913: epoch=971 step=971000 epoch=971 metrics={'time_inference': 0.00018904781341552734, 'time_environment_step': 1.2251853942871093e-05, 'time_step': 0.0002483832836151123, 'rollout_return': 200.0, 'time_sample_batch': 9.350776672363281e-05, 'time_algorithm_update': 0.0032314777374267576, 'loss': 3.2681543588638307, 'evaluation': 199.7} step=971000
2023-01-09 23:16.30 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_972000.pt
2023-01-09 23:16.30 [info     ] DoubleDQN_online_20230109230913: epoch=972 step=972000 epoch=972 metrics={'time_inference': 0.00018679213523864745, 'time_environment_step': 1.2262821197509765e-05, 'time_step': 0.0002442426681518555, 'rollout_return': 200.0, 'time_sample_batch': 8.265972137451172e-05, 'time_algorithm_update': 0.0030710935592651

2023-01-09 23:16.38 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_987000.pt
2023-01-09 23:16.38 [info     ] DoubleDQN_online_20230109230913: epoch=987 step=987000 epoch=987 metrics={'time_inference': 0.00019059658050537108, 'time_environment_step': 1.2299537658691406e-05, 'time_step': 0.00024830198287963866, 'rollout_return': 200.0, 'time_sample_batch': 8.296966552734375e-05, 'time_algorithm_update': 0.0030692338943481444, 'loss': 2.982913863658905, 'evaluation': 176.7} step=987000
2023-01-09 23:16.38 [info     ] Model parameters are saved to d3rlpy_logs/DoubleDQN_online_20230109230913/model_988000.pt
2023-01-09 23:16.38 [info     ] DoubleDQN_online_20230109230913: epoch=988 step=988000 epoch=988 metrics={'time_inference': 0.00020835351943969727, 'time_environment_step': 1.3186216354370118e-05, 'time_step': 0.000272294282913208, 'rollout_return': 200.0, 'time_sample_batch': 0.0001146554946899414, 'time_algorithm_update': 0.00347986221313476